In [1]:
import numpy as np
from numpy import vstack
import string
import matplotlib.pyplot as plt
import plotly as py
import plotly.graph_objs as go
print(py.__version__)
from collections import defaultdict

4.7.1


In [24]:
class HilberCurveGenerator:
    def __init__(self):
        self.space_alphabet = 'ABCDEFGH'
        self.recurrence_matrix = np.array([[0, 3, 4, 7, 6, 5, 2, 1],
                                           [0, 7, 6, 1, 2, 5, 4, 3],
                                           [0, 7, 6, 1, 2, 5, 4, 3],
                                           [2, 3, 0, 1, 6, 7, 4, 5],
                                           [2, 3, 0, 1, 6, 7, 4, 5],
                                           [4, 3, 2, 5, 6, 1, 0, 7],
                                           [4, 3, 2, 5, 6, 1, 0, 7],
                                           [6, 5, 2, 1, 0, 3, 4, 7]
                                           ]
                                          )
        self.quad_mappings = {'A': np.array([-1.0, -1.0, -1.0]),
                              'B': np.array([-1.0, 1.0, -1.0]),
                              'C': np.array([-1.0, 1.0, 1.0]),
                              'D': np.array([-1.0, -1.0, 1.0]),
                              'E': np.array([1.0, -1.0, 1.0]),
                              'F': np.array([1.0, 1.0, 1.0]),
                              'G': np.array([1.0, 1.0, -1.0]),
                              'H': np.array([1.0, -1.0, -1.0])
                              }

    def __recurse(self, base_array=None):
        if not base_array:
            base_array = self.space_alphabet

        next_results = []
        # Initial Run?

        if type(base_array) == str:
            base_array = np.array(list(base_array))
            # use idx to lookup reorder opperation to perform of base_array
            for idx, quad in enumerate(base_array):
                reorder_array = self.recurrence_matrix[idx]
                reordered_base = base_array[reorder_array]
                next_result = quad + '|' + ''.join(reordered_base)
                next_results.append(next_result)
        else:
            for quad_data in base_array:
                quad_base, base_array = quad_data.split('|')
                reordered_bases = self.__recurse(base_array=base_array)
                _ = [next_results.append(quad_base + reordered_base)
                     for reordered_base in reordered_bases]
        return next_results

    def __expand_last_recursion(self, quad_values):
        expanded_quads = []
        for quad_data in quad_values:
            quad_base, base_array = quad_data.split('|')
            _ = [expanded_quads.append(quad_base + sub_quad)
                 for sub_quad in base_array]
        return expanded_quads

    def generate_curve(self, plot_depth, return_plot_data=True):
        trace_data = defaultdict(lambda: defaultdict(lambda: []))

        recursed_data = None

        for depth in range(1, plot_depth):
            if not recursed_data:
                recursed_data = self.__recurse(base_array=self.space_alphabet)
            else:
                recursed_data = self.__recurse(base_array=recursed_data)

            quad_labels = self.__expand_last_recursion(recursed_data)

        return quad_labels


if __name__ == '__main__':
    hc_gen = HilberCurveGenerator()
    hc_seq = hc_gen.generate_curve(plot_depth=3)
    print(hc_seq)

['AAA', 'AAH', 'AAG', 'AAB', 'AAC', 'AAF', 'AAE', 'AAD', 'ADA', 'ADB', 'ADC', 'ADD', 'ADE', 'ADF', 'ADG', 'ADH', 'AEA', 'AEB', 'AEC', 'AED', 'AEE', 'AEF', 'AEG', 'AEH', 'AHE', 'AHH', 'AHA', 'AHD', 'AHC', 'AHB', 'AHG', 'AHF', 'AGE', 'AGH', 'AGA', 'AGD', 'AGC', 'AGB', 'AGG', 'AGF', 'AFG', 'AFH', 'AFE', 'AFF', 'AFC', 'AFD', 'AFA', 'AFB', 'ACG', 'ACH', 'ACE', 'ACF', 'ACC', 'ACD', 'ACA', 'ACB', 'ABC', 'ABF', 'ABE', 'ABD', 'ABA', 'ABH', 'ABG', 'ABB', 'BAA', 'BAB', 'BAC', 'BAD', 'BAE', 'BAF', 'BAG', 'BAH', 'BHA', 'BHD', 'BHE', 'BHH', 'BHG', 'BHF', 'BHC', 'BHB', 'BGA', 'BGD', 'BGE', 'BGH', 'BGG', 'BGF', 'BGC', 'BGB', 'BBG', 'BBB', 'BBA', 'BBH', 'BBE', 'BBD', 'BBC', 'BBF', 'BCG', 'BCB', 'BCA', 'BCH', 'BCE', 'BCD', 'BCC', 'BCF', 'BFC', 'BFB', 'BFG', 'BFF', 'BFE', 'BFH', 'BFA', 'BFD', 'BEC', 'BEB', 'BEG', 'BEF', 'BEE', 'BEH', 'BEA', 'BED', 'BDE', 'BDF', 'BDG', 'BDH', 'BDA', 'BDB', 'BDC', 'BDD', 'CAA', 'CAB', 'CAC', 'CAD', 'CAE', 'CAF', 'CAG', 'CAH', 'CHA', 'CHD', 'CHE', 'CHH', 'CHG', 'CHF', 'CHC'

In [25]:
def hilbert_index(seq):
    '''
    Converts input sequence into matrix index 
    for space filling curve in 4 quadrants ABCD
    
    Parameters:
    ----------
    seq: String
        Input string to be converted 
        
    Returns
    -------
    index: list
        List of x,y matrix indices 
    '''
    rules = {'A': np.array([-1, -1, -1]),
             'B': np.array([-1, 1, -1]),
             'C': np.array([-1, 1, 1]),
             'D': np.array([-1, -1, 1]),
             'E': np.array([1, -1, 1]),
             'F': np.array([1, 1, 1]),
             'G': np.array([1, 1, -1]),
             'H': np.array([1, -1, -1])
            }
    dim = 2
    index = np.array([])
    
    for order, val in enumerate(seq[::-1].upper()):
        if index.size == 0:
            index = np.zeros_like(rules[val])
        index += (dim ** order) * rules[val] 
    
    return index.tolist()

In [27]:
order = 3
coordinates = []
for i, seq in enumerate(hc_seq):
    coordinates.append(hilbert_index(seq))

x_vals = [(x*2+1)/(2**(order+1)) for [x,y,z] in coordinates]
y_vals = [(y*2+1)/(2**(order+1)) for [x,y,z] in coordinates]
z_vals = [(z*2+1)/(2**(order+1)) for [x,y,z] in coordinates]

In [28]:
h_i = hilbert_index('che')
h_x = h_i[0]
h_y = h_i[1]
h_z = h_i[2]

In [64]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x = x_vals,
    y = y_vals,
    z = z_vals,
    name='hilbert-3d',
    mode='lines',
    marker=dict(
        size=3,
        color=x_vals,
        colorscale='Viridis'),
    line=dict(
        color=z_vals,
    colorscale='Viridis',
    width=3),
))

fig.add_trace(go.Scatter3d(x = [(h_x*2+1)/(2**(order+1))],
                           y = [(h_y*2+1)/(2**(order+1))],
                           z = [(h_z*2+1)/(2**(order+1))],
                           name='point',
                           mode='markers',
                           marker=dict(
                               size=5,
                               color='#e377c2')))

fig.show()

In [65]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x = [0,0,0,0,1,1,1,1],
                         y = [0,1,1,0,0,1,1,0],
                         z = [0,0,1,1,1,1,0,0],
                         name='hilbert-curve-3d',
                         mode='lines+markers'))

fig.update_layout(
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    ),
    xaxis = dict(
        range=(0, 2),
        constrain='domain'
    )
)
fig.show()